링크 : [Neural Granger Causality](https://arxiv.org/abs/1802.05842)

### 1. 요약

`-` 목표

* 비선형 시계열 데이터에서 `Granger Causality`를 추론하기 위해, **신경망 기반 모델**(MLP,LSTM)을 사용하고, `sparsity penalty`(희소성 유도 정규화)를 도입한 새로운 방법을 제안

`-` 주요 내용

* **Granger causality**는 어떤 변수의 과거가 다른 변수의 미래를 예측하는데 도움이 되는지 보는 것
* 기존의 대부분의 방법은 **선형 모델(VAR)** 사용 $\to$ 비선형관계 잡지 못함
* **Component-wise neural nets**(각 출력 변수마다 독립적인 MLP 또는 LSTM) + **structured sparsity penalty**를 통해 해석 가능한 인과 구조를 학습
* 특히 **Group Lasso, Hierarchical Group Lasso** 등을 통해 특정 변수의 영향력 제거
* DREAM3 유전자 네트워크, 인간 동작 데이터 등에 적용하여 뛰어난 성능을 보임

### 2. Linear Granger causality

`-` Granger causality : 한 시계열의 과거 값이 다른 시계열이 미래 값을 예측하는데 도움이 되는지를 평가하는 개념

`-` VAR(Vector Autogressive Regression)모델

* Granger causality를 분석하는 가장 기본적인 방법
* 시계열 데이터 표현 $x_t=\sum_{k=1}^{K}A^{(k)}x_{t-k}+\epsilon_t$
* $x_{t}$ : 시점 t에서의 다변량 시계열 데이터(p개의 변수 포함)
* k : 최대 시차
* $A^{(k)}$ : 시차 k에서의 계수행렬(pxp크기)
* $e_t$ : 평균이 0인 잡음

`-` 인과성 판단 기준 - 특정 시계열 j가 다른 시계열 i의 미래를 예측하는데 기여하는지를 확인

* $A^k_{ij}=0$이면, 시계열 j는 시계열 i에 대해 Granger 비인과적
* $A^k_{ij}\neq 0$이면, 시계열 j는 시계열 i에 대해 Granger 인과적

`-` 희소성 유도 패널티 적용

* 너무 많은 변수를 고려하면 과적합문제 발생->Lasso,Group Lasso 패널티를 적용하여 불필요한 계수 0으로
    * Group Lasso 패널티 : 결과적으로 희소한 Granger 인과 네트워크를 학습
        * $min_{A(1),...,A(K)}\sum_{t=K}^{T}||x_t-\sum_{k=1}^{K}A^{(k)}x_{t-k}||^2_2+\lambda\sum_{ij}||(A^{(1)}_{ij},...,A^{K}_{ij}||_2,$
    * 계층적 패널티 : 너무 긴 시차를 사용하면 과적합이 발생할 수 있으므로 모델이 적절한 시차로 학습해아함

`-` VAR은 선형적 관계에서만 가정, 비선형 데이터는 한계

* $\to$ Neural Granger Causality 기법 필요

### 3. Models for Neural Granger Causality

#### 3.1 Adapting Neural Networks for Granger Causality
`-` **비선형 자기 회귀 모델(Nonlinear Autogressive Model, NAR)**

* **NAR** : 기존의 선형 VAR보다 훨씬 유연하게 과거-현재 관계 표현가능
* $x_t=g(x_{<t1},...,x_{<tp})+\epsilon_t$
* $x_t$ : 시점 t에서의 다변량 시계열 데이터
* $x_{<t1},...,x_{<tp}$ : 과거 p개의 시점에서 수집된 데이터
* $g(*)$ : **비선형 함수**(ex: 신경망)
* $e_t$ : 평균이 0인 잡음(noise)

`-` Problem

* 블랙박스 문제 : MLP/LSTM은 강력한 예측성능을 가지지만 전체 시계열을 한번에 모델링(출력 $x_t$를 하나의 신경망으로 학습) $\to$ 특정 입력 $x_j$가 특정 출력 $x_i$에 어떤 영향을 주는지 분리 어려움(숨겨진 층들을 공유하기 때문)
* 모든 출력이 같은 **시간지연(lag)**에 의존한다고 가 : 전통적인 VAR모델에서는 최대 시차 k를 설정, 일반적 NAR 딥러닝 모델에서는 입력으로 같은 시간지연만 넣음
    * 모든 시계열이 지난 3시점만 (t-1, t-2, t-3)
    * 하지만 어떤 변수는 짧은 lag, 어떤 변수는 긴 lag에 의존할 수 있음
    * $\to$ 현실적인 구조 반영 못함

`-` 해결방법

* cMLP,cLSTM : Component-wise Neural Network
* 각 출력 변수 $x_{ti}$마다 독립적인 신경망 모델 $g_i$를 사용하여 입력변수와의 관계를 학습
    * 출력을 하나하나 분리해서 학습
    * 이러한 구조를 component-wise architecture(그래서 모델 이름 cMLP,cLSTM)
* 장점 : 입력 시계열이 어떤 출력에 영향을 주는지 쉽게 해석 가능, 각 $g_i$함수에만 집중하면 되기 때문에 인과 추론이 쉬워짐

`-` Granger Non-Causality

* 시계열 j가 시계열 i의 Granger 원인이 아니다 $\leftrightarrow$ $g_i$ 함수가 $x_j$의 과거에 전혀 의존하지 않음
* $g_i(x_{t1},...,x_{tj},...x_{tp}) = g_i(x_{<ti},...,x^{'}_{<tj},...,x_{tp})$
* 여기서 $x_{<,tj} \neq x^{'}_{tj}$임에도 결과는 같음
* 즉, $x_j$의 과거가 바뀌어도 $x_{ti}$ 예측에 아무 영향이 없음

#### 3.2 Sparse Input MSPs(cMLP)

`-` 구조

* 각 출력 시계열 $x_i$ 에 대해 별도의 MLP $g_i$ 학습
* **희소성(sparsity)** 을 유도해서 관련 없는 입력은 가중치를 0으로
* 각 출력변수 $x_{ti}$는 다음과 같이 예측
    * $x_{ti}$ = $g_i(x_{<t})+\epsilon_{ti}$
    * $g_i(*)$ : MLP 함수
    * 입력 : 시간 지연 K 만큼 고려된 전체 시계열의 과거 $x_{t-1}, x_{t-2},...x_{t-k}$
    * 출력 : 하나의 시계열 값 $x_ti$
* L개의 층(은닉층 L-1개, 출력층 1개)
* 첫 번째 은닉층의 가중치가 가장 중요함 $\to$ 여기에 인과 구조가 반영
* 첫 층의 가중치 $W_1$은 시간 지연별로 나눔
    * $W^1={W^{1,1}, W^{1,2}, ..., W^{1,K}}$
    * 지연 k별 가중치 $W^{1,k}$ $\to$ lag별 효과 분석 가능

`-` Granger causality 판별 기준

* 입력 $x_j$의 과거값들이 출력 $x_i$에 영향을 주지 않는다면, 첫 층의 해당 입력에 연결된 모든 가중치가 0이어야함
    * $W^1_{:,j}=0$ for all lags $\to$ 시계열 j는 i의 Granger 원인이 아님
    * 그래서 가중치 0으로 만드는 `penalty` 도입!
* Group Lasso : 입력 시계열 j의 모든 지연에 대한 가중치를 하나의 그룹으로 보고, 그룹 전체를 0으로 만듦 $\to$ 시계열 j 전체를 삭제할지 말지 결정
* Sparse Group Lasso
    * 그룹간 sparsity + 그룹 내 sparsity 둘 다 유도
    * 일부 시계열은 완전 제거, 일부는 특정 lag만 유지
    * $\alpha$ : 전체 sparsity vs lag 선택간의 조절 파라미터
* Hierarchical Group Lasso
    * 특정 lag 이상은 모두 제거되도록 하는 penalty
    * lag 선택이 자연스럽게 이루어짐, 가중치 0인 관련없는 lag 자동으로 제거
    * $\to$ lag를 크게 설정해도 됨


#### 3.3 Sparse Input RNNs

`-` cLSTM

* 입력 : 전체 시계열 $x_t ∈R^p$
* 출력 : 오직 한 변수 $x_{ti}$ 만 예측하는 신경망 $g_i$
* $f_t = \sigma(W_fx_t+U_fh_{t-1})$(forget gate)
* $i_t = \sigma(W_inx_t+U_{in}h_{t-1}$(input gate)
* $o_t = \sigma(W_ox_t+U_oh_{t-1})$(output gate)
* $C_t=f_t \odot C_{t-1}+i_t \odot tanh(W_cx_t + U_ch_{t-1})$
* $h_t=o_t\odot tanh(C_t)$
* $C_t$ : 셀 상태
* $x_t$ : 현재 입력(전체 시계열)
* $h_t$ : 현재 hidden state
* $W_f, W_in, W_o,W_c$ : 입력 가중치 행렬들
* **인과 관계는 이 입력 가중치들(W)을 통해서 표현됨!!**

`-` Granger causality 추론 방법

* 시계열 j가 출력 $x_{ti}$에 영향을 주지 않으려면 입력 자중치 행렬 W의 j번째 column이 모두 0이 되면 됨
    * $W_{:,j}=0$ $\to$ 시계열 j는 i의 Granger 원인이 아님
* Group Lasso penalty 사용


`-` cMLP 와 cLSTM 차이

* cMLP
    * 고정된 lag(지정 필요)
    * 계산구조 - 완전 연결 구조
    * 첫 층 가중치로 인과 해석
* cLSTM
    * 장기 기억 자동 처리(lag지정 필요없음)
    * 계산구조 - 순환 구조
    * 입력 가중치 W로 인과 해석

### 4. Optimizing the Penalized Objectives

#### 4.1 Optimizing the Penalized cMLP Objective

`-` 최적화 방법 : Proximal Gradient Descent

* 희소성 penalty 미분 불가능 or , 정확이 0이되는 가중치 만들기 어려움 $\to$ sparsity를 정확히 유도하기 어려움
* Proximal Gradient는 `가중치가 딱 0이 되도록` 업데이트
* 알고리즘 구조
    * 현재 가중치 $W^m$에 대해 손실함수 $L(W)$의 gradient 계산
    * gradient만큼 이동
    * proximal 연산(prox)으로 sparsity penalty를 반영한 업데이트
* prox 연산은 각 penalty종류에 따라 다르게 정의
    * Group Lasso : column vector 전체를 soft-Thresholding
    * Sparse Group Lasso : 먼저 개별 lag별 soft-threshold $\to$ 전체 vector에 soft-threshold
    * Hierarchical Group Lasso : lag가 높은 그룹부터 차례대로 soft-threshold적용

#### 4.2 Optimizing the Penalized cLSTM Objective

`-` 최적화 방법 : 마찬가지로 Proximal Gradient Descent

`-` 구체적 학습 방법

* Backpropagation Through Time(BPTT)로 gradient 계산
    * LSTM은 시간 축으로 펼쳐야 하니까 이걸 사용
    * 시계열 길이가 길경우에 truncated BPTT사용 $\to$ 길게 펼치지 않고 적당히 잘라서 계산
* 학습 알고리즘
    * 현재 가중치 $W^m$에 대해 gradient 계산
    * step size $\gamma$ 결정
    * 입력가중치 $W^1_{:,j}$ 에 대해 soft-threshold적용
    * 나머지 파라미터(출력층 $W^2$, 은닉 상태 관련 $U$)는 일반 gradient 업데이트

### 5. Comparing cMLP and cLSTM Models for Granger Causality

`-` `lag`(지연 시간) 처리 방식

* cMLP
    * 최대 `lag` K를 명시적으로 지정
    * 하지만 hierarchical penalty 덕분에 불필요한 lag는 자동 제거
* cLSTM
    * lag지정할 필요 없음
    * LSTM 내부 구조가 장기 의존성 자동 처리

`-` 사용 가능한 데이터량

* cMLP
    * 최대 lag `K` 때문에 훈련에 사용할 수 있는 데이터 수가 `T-K`
* cLSTM
    * cLSTM은 한 시점 전부터 사용 가능하므로 T-1ro epdlxj tkdyd rksmd
    * $\to$ 작은 데이터셋에서는 cLSTM이 상대적으로 더 많은 학습데이터 활용 가능

`-` 희소성의 의미

* 둘 다 입력 가중치에 정규화를 걸어 Granger 인과성 없음 $\to$ 가중치 0 유도

`-` 한계

* cMLP
    * lag지정 잘못되면 성능 저하
* cLSTM
    * 해석력은 좋으나 내부 구조 해석은 다소 복잡

### 6. Simulation Experiments

#### 6.1 cMLP and cLSTM Simulation Comparison

`-` 목적

* 두 모델이 선형 및 비선형 시계열에서 Granger 인과 그래프를 얼마나 잘 복원하는지
* 사용한 시뮬레이션 데이터
    * Lorenz-95 모델 : 비선형, 복잡한 시스템
    * VAR(Vector AutoRegressive)모델 : 선형구조

##### 6.1.1 Lorenz-96 모델 실험

* 비선형적 다변량 동적 시스템
* 기후 모델링 등에 사용
* $\frac{dx_{ti}}{dt}=(x_{t(i+1)}-x_{t(i-2)})x_{t(i-1)}-x_{ti}+F$
* F : 혼란스러움 조절 파라미터 $F∈\{10,40\}$
    * F=10 : 덜 복잡
    * F=40 : 더 chaotic
* 차원 $p=20$, 시계열 길이 $T∈\{250,500,1000\}$
* 각 설정마다 5번 반복하고 평균을 냈음
| Model          | T=250 | T=500 | T=1000 |
|----------------|-------|-------|--------|
| cMLP (F=10)    | 86.6  | 96.6  | 98.4   |
| cLSTM (F=10)   | 81.3  | 93.4  | 96.0   |
| cMLP (F=40)    | 84.0  | 89.6  | 95.5   |
| cLSTM (F=40)   | 75.1  | 87.8  | 94.4   |
* 전반적으로 **cMLP**가 높은 정확도 **(AUROC)-Grangr 인과구조 복원 정확도** 를 보임
* 데이터가 많아질수록 성능이 비슷해짐

##### 6.1.2 VAR 모델 실험

* 선형 시계열 모델, 각 시계열이 다른 시계열들의 과거값에 선형적으로 영향을 받음
* $x_t = A_1x_{t-1}+A_2x_{t-2}+...+A_kx_{t-k}+\epsilon_t$
* $A_k$ : 계수 행렬
* Ground truth Granger 구조가 정확히 알려져 있는 상황에서 테스트 가능
* 차원 $p=10$, 시계열 길이 $T∈\{250,500,1000\}$
* lag K=3
| Model          | T=250 | T=500 | T=1000 |
|----------------|-------|-------|--------|
| cMLP     | 94.6  | 98.7  | 99.5   |
| cLSTM    | 91.1  | 96.6  | 99.2   |
* 둘다 매우 높은 정확도
* 적은 데이터일수록 cMLP가 안정적으로 좋음

`-` 해석

* VAR은 선형 구조이므로 cMLP가 더 적합한 구조
* LSTM은 비선형성 + 복잡한 Long-term 구조 학습에 강하지만 단순 선형 시계열에서는 오버피팅이 일어날 수도 있음
* 결론 : Granger인과 구조가 명확하고 선형적인 경우 **cMLP가 더 정확하고 빠르게 복원가능**

#### 6.2 Real Motion Capture Data

`-` 목적

* 

---